In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import kagglehub

In [2]:
pip install nltk

In [4]:
# Base de dados
path = kagglehub.dataset_download("sachinkumar62/movies-details")
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
print("Path to dataset files:", path)
os.listdir(path)

100%|██████████| 1.21M/1.21M [00:00<00:00, 88.2MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sachinkumar62/movies-details/versions/1


['movies.csv']

In [5]:
path = '/root/.cache/kagglehub/datasets/sachinkumar62/movies-details/versions/1'
df = pd.read_csv(path + '/movies.csv')
df.head()

,Unnamed: 0,id,title,overview,release_date,popularity,vote_average,vote_count
0,0,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-20,18.433,8.7,2763
1,1,724089,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,2020-07-31,8.439,8.7,1223
2,2,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,65.570,8.7,18637
3,3,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,63.277,8.7,14052
4,4,761053,Gabriel's Inferno Part III,The final part of the film adaption of the ero...,2020-11-19,26.691,8.7,773


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8551 entries, 0 to 8550
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8551 non-null   int64  
 1   id            8551 non-null   int64  
 2   title         8551 non-null   object 
 3   overview      8531 non-null   object 
 4   release_date  8551 non-null   object 
 5   popularity    8551 non-null   float64
 6   vote_average  8551 non-null   float64
 7   vote_count    8551 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 534.6+ KB


In [7]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
# Baixar recursos do NLTK
nltk.data.path.append('/kaggle/input/nltk-data')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Função para limpar o texto
def preprocess_text(text):
    # Transformar o texto para minúsculas
    text = text.lower()

    # Tokenizar o texto
    tokens = word_tokenize(text)

    # Remover stopwords e pontuação, e lematizar
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]

    # Reconstruir o texto
    return ' '.join(tokens)

In [11]:
# Função para limpar o texto
def preprocess_text(text):
    # Transformar o texto para minúsculas
    text = text.lower()

    # Tokenizar o texto
    tokens = word_tokenize(text)

    # Remover stopwords e pontuação, e lematizar
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]

    # Reconstruir o texto
    return ' '.join(tokens)

In [16]:
# Remover linhas com valores ausentes na coluna 'overview'
df_clean = df.dropna(subset=['overview'])

# Criar o modelo TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustar e transformar o texto
tfidf_matrix = tfidf_vectorizer.fit_transform(df_clean['overview'])

# Verificar o formato da matriz
print(tfidf_matrix.shape)  # Isso deve mostrar a quantidade de filmes e o número de características

(8531, 26613)


In [17]:
# Calcular a similaridade cosseno entre todos os filmes
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Verificar a similaridade entre os primeiros 3 filmes
print(cosine_sim[:3, :3])

[[1.         0.04820789 0.06266735]
 [0.04820789 1.         0.0470334 ]
 [0.06266735 0.0470334  1.        ]]


In [18]:
def recommend_movies(movie_title, cosine_sim=cosine_sim):
    # Obter o índice do filme
    idx = df[df['title'] == movie_title].index[0]

    # Obter os scores de similaridade para o filme
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar os filmes com base na similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obter os índices dos filmes mais similares
    movie_indices = [i[0] for i in sim_scores[1:6]]  # Pegue os 5 filmes mais similares

    # Retornar os títulos dos filmes recomendados
    return df['title'].iloc[movie_indices]

# Testar com um título de filme
recommended_movies = recommend_movies('The Matrix')
print(recommended_movies)


5679                          The World Is Not Enough
2236                                    Kung Fu Panda
1705                                    Becoming Jane
8430    In the Name of the King: A Dungeon Siege Tale
4212                                   Finding 'Ohana
Name: title, dtype: object


In [20]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular a similaridade de cosseno entre todos os filmes
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Função para recomendar filmes com base no índice do filme
def recommend_movie(index, cosine_sim=cosine_sim):
    movie_title = df['title'].iloc[index]

    # Obter os índices dos filmes mais semelhantes
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Pegar os 10 filmes mais semelhantes (excluindo o próprio filme)
    sim_scores = sim_scores[1:11]

    # Obter os índices dos filmes
    movie_indices = [i[0] for i in sim_scores]

    # Retornar os títulos dos filmes recomendados
    print(f"Recomendações para {movie_title}:")
    return df['title'].iloc[movie_indices]

# Teste: recomendar filmes similares ao filme de índice 0 (primeiro filme)
recommended_movies = recommend_movie(0)
print(recommended_movies)


Recomendações para Dilwale Dulhania Le Jayenge:
2121                                          Trumbo
8491                                     Piranha 3DD
485                     The Bridge on the River Kwai
1690                                        Precious
7665                         Masters of the Universe
7424                          Love Lasts Three Years
3687           The Little Mermaid: Ariel's Beginning
3030    Belle and Sebastian: The Adventure Continues
975                                   Instant Family
4316                                   Project Power
Name: title, dtype: object
